In [1]:
import cv2
import mediapipe as mp
import numpy as np
import time


In [2]:
mp_pose = mp.solutions.pose
mp_drawing = mp.solutions.drawing_utils


In [4]:
def calculate_angle(a, b, c):
    radians = np.arctan2(c.y - b.y, c.x - b.x) - \
              np.arctan2(a.y - b.y, a.x - b.x)
    angle = np.abs(radians * 180.0 / np.pi)

    if angle > 180:
        angle = 360 - angle

    return round(angle, 2)


In [5]:
PLANK_MIN_ANGLE = 165
PLANK_MAX_ANGLE = 180

plank_status = "Not Ready"
plank_start_time = None
hold_time = 0


In [9]:
cap = cv2.VideoCapture(0)
cap = cv2.VideoCapture(0)
cap.set(cv2.CAP_PROP_FRAME_WIDTH, 640)
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 480)
frame_count = 0



In [11]:
with mp_pose.Pose(min_detection_confidence=0.5,
                  min_tracking_confidence=0.5) as pose:

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        frame_count += 1
        if frame_count % 2 != 0:
            continue  # پردازش هر فریم دوم → سرعت بالاتر

        # BGR -> RGB
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False
        results = pose.process(image)

        # RGB -> BGR
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

        if results.pose_landmarks:
            landmarks = results.pose_landmarks.landmark

            # نقاط مورد نظر
            shoulder = landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value]
            hip = landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value]
            ankle = landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE.value]

            # محاسبه زاویه بدن
            radians = np.arctan2(ankle.y - hip.y, ankle.x - hip.x) - \
                      np.arctan2(shoulder.y - hip.y, shoulder.x - hip.x)
            body_angle = np.abs(radians * 180.0 / np.pi)
            if body_angle > 180:
                body_angle = 360 - body_angle
            body_angle = round(body_angle, 2)

            # بررسی فرم صحیح
            if PLANK_MIN_ANGLE <= body_angle <= PLANK_MAX_ANGLE:
                plank_status = "Correct Plank"
                color = (0, 255, 0)
                if plank_start_time is None:
                    plank_start_time = time.time()
                hold_time = time.time() - plank_start_time
            else:
                plank_status = "Incorrect Plank"
                color = (0, 0, 255)
                plank_start_time = None
                hold_time = 0

            # نمایش زاویه
            h, w, _ = frame.shape
            hip_coords = tuple(np.multiply((hip.x, hip.y), [w, h]).astype(int))
            cv2.putText(image, f"Angle: {body_angle}", hip_coords,
                        cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255), 2)

            # رسم اسکلت سبک
            mp_drawing.draw_landmarks(
                image,
                results.pose_landmarks,
                mp_pose.POSE_CONNECTIONS,
                mp_drawing.DrawingSpec(color=(245,117,66), thickness=1, circle_radius=2),
                mp_drawing.DrawingSpec(color=(245,66,230), thickness=1, circle_radius=2)
            )

        # نمایش وضعیت و زمان نگه‌داشتن
        cv2.rectangle(image, (0,0), (350,100), color, -1)
        cv2.putText(image, "PLANK STATUS", (10,30),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0,0,0), 2)
        cv2.putText(image, plank_status, (10,65),
                    cv2.FONT_HERSHEY_SIMPLEX, 1.0, (255,255,255), 2)
        cv2.putText(image, f"Time: {int(hold_time)}s", (10,95),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255,255,255), 2)

        # نمایش فریم
        cv2.imshow("Plank Form Evaluation", image)

        if cv2.waitKey(10) & 0xFF == ord('q'):
            break


KeyboardInterrupt: 

In [12]:
cap.release()
cv2.destroyAllWindows()